# PixelByte: Catching Insights in Unified Multimodal Sequences

Ce notebook présente **PixelByte**, un modèle innovant conçu pour générer simultanément du texte et des images pixel par pixel sous forme de séquences. L'objectif est d'explorer un embedding unifié qui permet une génération multimodale cohérente.

## Contexte et Architecture Proposée

### Fondements Théoriques
- **Image Transformer** : [Génération d'images pixel par pixel](https://arxiv.org/abs/1802.05751)
- **Bi-Mamba+** : [Modèle bidirectionnel pour la prévision de séries temporelles](https://arxiv.org/abs/2404.15772)
- **MambaByte** : [Modèle d'état d'espace sélectif sans token](https://arxiv.org/abs/2401.13660)

### Concept Clé
Le modèle PixelByte génère des séquences mixtes de texte et d'images. Il doit :
- Gérer les transitions entre texte et image avec des sauts de ligne (ASCII 0A).
- Maintenir la cohérence des dimensions des images générées.
- Assimiler la tâche de "copie" pour reproduire des motifs complexes.

Ce notebook utilise la puissance des GPU T4 x2 de Kaggle pour expérimenter avec des architectures avancées et des jeux de données volumineux, afin de relever les défis de la génération multimodale unifiée.

## Ressources du Projet

### Dataset
Pour ce projet, nous utiliserons le dataset **PixelBytes-Pokemon**, spécialement conçu pour cette tâche de génération multimodale. Ce dataset, créé par l'auteur de ce notebook, est disponible sur Hugging Face : [PixelBytes-Pokemon](https://huggingface.co/datasets/ffurfaro/PixelBytes-Pokemon). Il contient des séquences de texte et d'images de Pokémon, encodées de manière à permettre l'entraînement de notre modèle PixelByte sur des données multimodales.

### Implémentation
L'implémentation du modèle et les scripts d'entraînement sont disponibles dans le dépôt GitHub **Mamba-Bys** : [Mamba-Bys](https://github.com/fabienfrfr/Mamba-Bys). Ce dépôt contient le code source nécessaire pour reproduire les expériences, ainsi que des instructions détaillées sur la configuration et l'utilisation du modèle PixelByte.

In [1]:
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

## à tester 

- le calcul de la fonction de cout et de la precision (bizarre qu'elle soit à 40 % des le debut)
- l'embedding PxByEmbed creer pour notre modele ! il y a des incoherences et trop de mémoire alloué !


In [2]:
from pixelbytes.mambabys import *
from pixelbytes.train import *
from pixelbytes.dataset import *
from pixelbytes.tokenizer import *

from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/opt/conda/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/opt/conda/lib/python3.10/site-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/opt/conda/lib/python3.10/site-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [3]:
# init
hf_dataset = load_dataset("ffurfaro/PixelBytes-Pokemon")
ds = hf_dataset["train"].train_test_split(test_size=0.1)

train_dataset = PxByDataset(ds["train"]["pixelbyte"], seq_length=256, stride=32)
test_dataset = PxByDataset(ds["test"]["pixelbyte"], seq_length=256, stride=32)

pixelbyte = PixelBytesTokenizer()

Generating train split:   0%|          | 0/964 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
vocab_size = len(pixelbyte.vocab)
embedding_dim = 16
hidden_dim = 64
# choose model (simple, attention, mamba)
#model = SimpleSeqModel(vocab_size, embedding_dim, hidden_dim)
model = SimpleAttentionModel(vocab_size, embedding_dim, hidden_dim)
# Mamba
d_model = 64 # 256 is to hudge !
d_state = 16
n_layers = 8
config = MambaConfig(dim=d_model, d_state=d_state, depth=n_layers, vocab_size=vocab_size)
# import model
#model = BysMamba(config)

In [8]:
"""
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    batch_size=32,
    learning_rate=0.001,
    num_epochs=200,
    save_dir='model_checkpoints',
    compile_model=False, #True,
    model_name="SimpleSeqModel",
    dataset_name="PixelBytes-Pokemon",
    eval_every=5  # Évaluer tous les 5 epochs
)
"""

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    batch_size=32,
    learning_rate=0.001,
    num_epochs=200,
    save_dir='model_checkpoints',
    compile_model=False, #True,
    model_name="SimpleAttentionModel",
    dataset_name="PixelBytes-Pokemon",
    eval_every=5  # Évaluer tous les 5 epochs
)

"""
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    batch_size=2,
    learning_rate=0.001,
    num_epochs=200,
    save_dir='model_checkpoints',
    compile_model=False, #True,
    model_name="Mambabys",
    dataset_name="PixelBytes-Pokemon",
    eval_every=5  # Évaluer tous les 5 epochs
)
"""

trainer.train()

Epoch 1/200: 100%|██████████| 728/728 [00:31<00:00, 23.23it/s]


Epoch 1/200, Average Training Loss: 2.3358


Epoch 2/200: 100%|██████████| 728/728 [00:31<00:00, 23.16it/s]


Epoch 2/200, Average Training Loss: 1.9939


Epoch 3/200: 100%|██████████| 728/728 [00:31<00:00, 22.93it/s]


...

Epoch 108/200, Average Training Loss: 0.6785


Epoch 109/200: 100%|██████████| 728/728 [00:31<00:00, 22.88it/s]


Epoch 109/200, Average Training Loss: 0.6778


Epoch 110/200:  49%|████▊     | 354/728 [00:15<00:16, 22.80it/s]


KeyboardInterrupt: 